In [2]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, ReLU

#Set the path to your train dataset directory
dataset_path = './dataset/train'

# Define the image size for resizing
image_size = (32, 32)

# Load and preprocess the dataset
X_train = []
y_train = []

# Iterate through the shape directories in the dataset path
for shape_dir in os.listdir(dataset_path):
    shape_path = os.path.join(dataset_path, shape_dir)
    if os.path.isdir(shape_path):
        # Iterate through the images in each shape directory
        for image_file in os.listdir(shape_path):
            image_path = os.path.join(shape_path, image_file)
            if image_path.endswith('.jpg'):
                # Load the image and resize it to the desired size
                image = cv2.imread(image_path)
                image = cv2.resize(image, image_size)

                # Append the preprocessed image and corresponding label
                X_train.append(image)
                y_train.append(shape_dir)

# Convert the image data and labels to NumPy arrays
X_train = np.array(X_train)
y_train = np.array(y_train)

#Set the path to your test dataset directory
dataset_path = './dataset/test'

# Load and preprocess the dataset
X_test = []
y_test = []

# Iterate through the shape directories in the dataset path
for shape_dir in os.listdir(dataset_path):
    shape_path = os.path.join(dataset_path, shape_dir)
    if os.path.isdir(shape_path):
        # Iterate through the images in each shape directory
        for image_file in os.listdir(shape_path):
            image_path = os.path.join(shape_path, image_file)
            if image_path.endswith('.jpg'):
                # Load the image and resize it to the desired size
                image = cv2.imread(image_path)
                image = cv2.resize(image, image_size)

                # Append the preprocessed image and corresponding label
                X_test.append(image)
                y_test.append(shape_dir)

# Convert the image data and labels to NumPy arrays
X_test = np.array(X_test)
y_test = np.array(y_test)

# Perform train-test split: This has already been done in the dataset
#X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=42)

#Set the path to your Validation dataset directory
dataset_path = './dataset/val'

# Load and preprocess the dataset
X_val = []
y_val = []

# Iterate through the shape directories in the dataset path
for shape_dir in os.listdir(dataset_path):
    shape_path = os.path.join(dataset_path, shape_dir)
    if os.path.isdir(shape_path):
        # Iterate through the images in each shape directory
        for image_file in os.listdir(shape_path):
            image_path = os.path.join(shape_path, image_file)
            if image_path.endswith('.jpg'):
                # Load the image and resize it to the desired size
                image = cv2.imread(image_path)
                image = cv2.resize(image, image_size)

                # Append the preprocessed image and corresponding label
                X_val.append(image)
                y_val.append(shape_dir)

# Convert the image data and labels to NumPy arrays
X_val = np.array(X_val)
y_val = np.array(y_val)


# Encode the shape labels into numerical values
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)
y_test_encoded = label_encoder.transform(y_test)

# Convert the numerical labels to one-hot encoded vectors
num_classes = len(label_encoder.classes_)
y_train_one_hot = to_categorical(y_train_encoded, num_classes)
y_val_one_hot = to_categorical(y_val_encoded, num_classes)
y_test_one_hot = to_categorical(y_test_encoded, num_classes)

# Create the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64))
model.add(ReLU())
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train_one_hot, epochs=10, batch_size=32, validation_data=(X_val, y_val_one_hot))


Epoch 1/10
235/235 [==============================] - 8s 27ms/step - loss: 3.5947 - accuracy: 0.4792 - val_loss: 0.5986 - val_accuracy: 0.7692
Epoch 2/10
235/235 [==============================] - 5s 23ms/step - loss: 0.3565 - accuracy: 0.8661 - val_loss: 0.3437 - val_accuracy: 0.8836
Epoch 3/10
235/235 [==============================] - 6s 25ms/step - loss: 0.1406 - accuracy: 0.9475 - val_loss: 0.1606 - val_accuracy: 0.9420
Epoch 4/10
235/235 [==============================] - 6s 26ms/step - loss: 0.0647 - accuracy: 0.9757 - val_loss: 0.1418 - val_accuracy: 0.9588
Epoch 5/10
235/235 [==============================] - 7s 28ms/step - loss: 0.0426 - accuracy: 0.9847 - val_loss: 0.1043 - val_accuracy: 0.9672
Epoch 6/10
235/235 [==============================] - 6s 27ms/step - loss: 0.0356 - accuracy: 0.9868 - val_loss: 0.1502 - val_accuracy: 0.9516
Epoch 7/10
235/235 [==============================] - 7s 29ms/step - loss: 0.0281 - accuracy: 0.9904 - val_loss: 0.1039 - val_accuracy: 0.9652

error: OpenCV(4.6.0) C:\ci_311_rebuilds\opencv-suite_1679001454889\work\modules\imgproc\src\resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


# Check Predicted Shape

In [16]:
#Set the path to your test dataset directory
image_path = './dataset/new/'
# Define a new data point to classify
for i in range(10):
    j = i+1
    temp = os.path.join(image_path, 'newshape' +str(j)+'.jpg') #Current Image
    print(temp)
    
    new_data = np.array([cv2.resize(cv2.imread(temp), image_size)])

    # Predict the label for the new data point
    predicted_probabilities = model.predict(new_data)
    predicted_label = label_encoder.inverse_transform(np.argmax(predicted_probabilities, axis=1))

    # Print the predicted label
    print("Predicted shape:", predicted_label)

./dataset/new/newshape1.jpg
1/1 [==============================] - 0s 22ms/step
Predicted shape: ['triangle']
./dataset/new/newshape2.jpg
1/1 [==============================] - 0s 21ms/step
Predicted shape: ['circle']
./dataset/new/newshape3.jpg
1/1 [==============================] - 0s 19ms/step
Predicted shape: ['trapezoid']
./dataset/new/newshape4.jpg
1/1 [==============================] - 0s 21ms/step
Predicted shape: ['trapezoid']
./dataset/new/newshape5.jpg
1/1 [==============================] - 0s 26ms/step
Predicted shape: ['rectangle']
./dataset/new/newshape6.jpg
1/1 [==============================] - 0s 28ms/step
Predicted shape: ['triangle']
./dataset/new/newshape7.jpg
1/1 [==============================] - 0s 25ms/step
Predicted shape: ['trapezoid']
./dataset/new/newshape8.jpg
1/1 [==============================] - 0s 24ms/step
Predicted shape: ['triangle']
./dataset/new/newshape9.jpg
1/1 [==============================] - 0s 22ms/step
Predicted shape: ['rectangle']
./datas

In [3]:
# Evaluate the model on the test data
loss, accuracy = model.evaluate(X_test, y_test_one_hot)

# Print the accuracy
print("Accuracy:", accuracy)

79/79 [==============================] - 1s 7ms/step - loss: 0.1247 - accuracy: 0.9628
Accuracy: 0.9628000259399414
